<a href="https://colab.research.google.com/github/Jacob-McM/Neural-Network-Modelling/blob/main/alphabetsoup_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
## Upload File
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (1).csv


In [13]:
#  Import and read the charity_data.csv.
import pandas as pd 
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Step 1: Preprocess Data

In [ ]:
## Drop Identification Columns 
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head(1)

#### `APPLICATION_TYPE` column preprocessing

---



In [75]:
## Make workable dataframe out of `APPLICAITON_TYPE` column to use functions on to determine values and potential cut-off points & bins

app_counts = pd.DataFrame()
app_counts['count'] = application_df['APPLICATION_TYPE'].value_counts()
app_sum_counts = application_df['APPLICATION_TYPE'].value_counts().sum()
print(application_df['APPLICATION_TYPE'].value_counts())
print(f"Sum of all values: {application_df['APPLICATION_TYPE'].value_counts().sum()}")

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64
Sum of all values: 34299


In [76]:
app_counts['percentage'] = app_counts['count']/app_sum_counts*100
app_counts.reset_index(inplace=True)
app_counts

,index,count,percentage
0,T3,27037,78.827371
1,T4,1542,4.495758
2,T6,1216,3.545293
3,T5,1173,3.419925
4,T19,1065,3.105047
5,T8,737,2.148751
6,T7,725,2.113764
7,T10,528,1.539403
8,Other,276,0.804688


In [77]:
## Decided cut off will be app types that cover less than 1%. 
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

#### `CLASSIFICATION` column preprocessing

---



In [78]:
## Make workable dataframe out of `CLASSIFICATION` column to use functions on to determine values and potential cut-off points & bins

class_counts = pd.DataFrame()
class_counts['count'] = application_df['CLASSIFICATION'].value_counts()
class_sum_counts = application_df['CLASSIFICATION'].value_counts().sum()

print(application_df['CLASSIFICATION'].value_counts())
print(f"Sum of all values: {application_df['CLASSIFICATION'].value_counts().sum()}")

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64
Sum of all values: 34299


In [79]:
class_counts['percentage'] = class_counts['count']/class_sum_counts*100
class_cutoff = class_counts.loc[class_counts['percentage'] < 1].copy()
class_cutoff = class_cutoff.reset_index()
class_cutoff

,index,count,percentage


In [81]:
## Decided cut-off will be classification types that cover less than 1%

classifications_to_replace = class_cutoff['index'].values.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

#### Data conversion and final preprocessing steps

In [82]:
## Convert to numerical data
preprocess_df = pd.get_dummies(application_df)
preprocess_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
## Define Target values (y) and Features (X)
y = preprocess_df['IS_SUCCESSFUL']
X = preprocess_df.drop('IS_SUCCESSFUL', axis=1).values

## Split data in training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =7)

In [ ]:
## Scaler instance
scaler = StandardScaler()

## Fit StandardScaler
X_scaler = scaler.fit(X_train)

## Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 2: Compile, Train, and Evaluate the Model

In [53]:
## Define model. Get number of input features and derive hidden layers from that number. 
## As the input features list was 44, I chose half of that for the first hidden layer. For the second hidden layer I used half of the first hidden layer.
input_features = len(X_train[0])
hidden_layer_1 = 22
hidden_layer_2 = 11
outputs = 1

nn = tf.keras.models.Sequential()

## First Hidden Layer
nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Output Node
nn.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 22)                990       
                                                                 
 dense_13 (Dense)            (None, 11)                253       
                                                                 
 dense_14 (Dense)            (None, 1)                 12        
                                                                 
Total params: 1,255
Trainable params: 1,255
Non-trainable params: 0
_________________________________________________________________


In [54]:
## Compile Model

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
## Train Model. Goal accuracy = 75%
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5900 - accuracy: 0.6997
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7297
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7307
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accura

In [56]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5547 - accuracy: 0.7285 - 547ms/epoch - 2ms/step
Loss: 0.5546782612800598, Accuracy: 0.7285131216049194


In [59]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [64]:
tf.keras.Model.save(nn,filepath='AlphabetSoupCharity.h5')

In [66]:
# nn_load_test = tf.keras.models.load_model('/content/gdrive/MyDrive/keras_model/model_1.h5')

In [67]:
# model_loss, model_accuracy = nn_load_test.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5547 - accuracy: 0.7285 - 496ms/epoch - 2ms/step
Loss: 0.5546782612800598, Accuracy: 0.7285131216049194


First attempt reaches an accuracy of 72.85%. In order to reach or surpass 75% adjustments will be made.